In [19]:
%run NB_Image.ipynb

In [2]:
# go wide screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from __future__ import print_function
from astropy.coordinates import SkyCoord
from dustmaps.sfd import SFDQuery

s
import extinction
from extinction import ccm89

from astropy.table import Table, Column
import astropy.units as u

In [4]:
%config InlineBackend.close_figures=True 

In [5]:
from IPython.html import widgets
from ipywidgets import Layout
from IPython.display import display,clear_output

/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [6]:
import matplotlib.pyplot as plt

params = {'legend.fontsize': '16',
          'figure.figsize': (15, 5),
         'axes.labelsize': '28',
         'axes.titlesize':'28',
         'xtick.labelsize':'28',
         'ytick.labelsize':'28'}

plt.rcParams.update(params)

In [7]:
from astropy.io import ascii

c = ascii.read("../data/mmsf2outcube_COSMOSC_allifu.cat")

In [8]:
isplit_good = c["split_good"] == "True"

ipointsource = c["class2"] == "pointsource"

ishowcase = c["showcase"] 

ii = ipointsource * isplit_good

In [10]:
c[ii][:3]

ifu,id,N,counts,flux,ra_com,dec_com,ddec,dra,x_com,y_com,z_com,dx,dy,dz,sqrt(ev1),sqrt(ev2),size2d,x_ext,y_ext,z_ext,wl_com,dwl,xmin,xmax,ymin,ymax,zmin,zmax,class,manualx,manualy,manualz,class2,comment2,use_stack,split_good,showcase,xray_counterpart
,,px,counts,erg / (cm2 s),RA[J2000],Dec[J2000],arcsec,arcsec,px,px,px,px,px,px,px,px,px,px,px,px,A,A,px,px,px,px,px,px,,,,,,,,,,
str3,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,int64,int64,int64,int64,int64,int64,str14,float64,float64,float64,str11,str135,bool,object,bool,bool
021,10001,773,2.2536e+02,9.0865e-17,150.330209,2.259786,1.5,1.4,89.34,92.92,62.10,2.91,2.73,3.53,3.32,2.23,4.00,17,15,17,3616.08,33.76,81,98,86,101,52,69,N/A,91.0,94.0,62.0,pointsource,LAE,False,True,False,False
021,5006,193,9.6403e+01,1.9428e-17,150.329382,2.251960,1.0,0.7,95.29,36.57,245.19,2.08,1.45,1.29,2.10,1.42,2.54,8,6,5,3979.66,9.93,91,99,34,40,243,248,N/A,96.0,36.0,244.0,pointsource,N/A,True,True,False,False
021,5007,200,1.0009e+02,1.7523e-17,150.334247,2.254039,0.9,0.7,60.29,51.53,284.73,1.77,1.40,1.88,1.82,1.34,2.26,8,5,8,4058.19,15.89,56,64,49,54,281,289,N/A,60.0,51.0,284.0,pointsource,N/A,False,True,False,False


In [29]:
# Select prime example of filament for reference
field = "COSMOSC"
#jj  =  c["ifu"] == '025' 
#jj *=  c["id"] == 10018
#r = c[jj][0]

rbuttons = {}

width = 5.

for r in c[ii][:]:
    try:
        ifu =  r["ifu"]
        wlstart = r["wl_com"] - width/2.
        wlend = r["wl_com"] + width/2.
        ra,dec = r["ra_com"], r["dec_com"]
        x,y = r["x_com"], r["y_com"]
        vmin=0.
        vmax=2.

        fncube = "../data/outcube_{}_{}.fits.gz".format(field, ifu)
        fncal = "../specphot/mean_cal_{}.txt".format(field)

        f = plt.figure(figsize=[7,7])
        ax = plt.subplot(111)
        ticks = np.arange(vmin,vmax+.001,.5)
        show_nb_image(fncube, fncal, wlstart, wlend, \
                      vmin = vmin, vmax = vmax, x = x, y = y, ticks=ticks, ax=ax,\
                     marker='+', markersize=50, alpha=.5)
        plt.plot([1,2,3])
        plt.xlabel("RA")
        plt.ylabel("Dec")


        canvas  = widgets.Output(layout=Layout(border='light'))

        rbutton_good = widgets.RadioButtons(
            options=['unclear', 'yes', 'no'],
            description='Detected in split exposure:',
            disabled=False
        )

        rbuttons[(r["ifu"],r["id"])] = rbutton_good

        canvas  = widgets.Output(layout=Layout(border='light'))
        info = widgets.HTML(
            value="ifu:{}<br> id:{}".format( r["ifu"],r["id"] )
        )

        if "split_good" in c.colnames:
            if r["split_good"] == "True":
                rbutton_good.value = 'yes'
            elif r["split_good"] == "False":
                rbutton_good.value = 'no'
            else:
                rbutton_good.value = 'unclear'


        hb = widgets.HBox([info,canvas, rbutton_good], layout=Layout(width = '1500px', positioning="bottom"))

        with canvas:
            clear_output(wait=True)
            display(f) 

        display(hb)
    except:
        pass

/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/Users/mxhf/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:20: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [18]:
plt.plot?

In [30]:
from astropy.table import Column
cat = ascii.read("../data/mmsf2outcube_COSMOSC_allifu.cat")
if not "split_good" in cat.colnames:
    cat.add_column(Column([None]*len(cat), name="split_good", description="Source still detected in split cubes.", dtype="S4"))


print("Split detect")
for rb in rbuttons:
    ifu, id = rb
    print("{} {} {}".format(ifu, id, rbuttons[rb].value))
    jj = (cat["ifu"] == ifu) * (cat["id"] == id)     
    if rbuttons[rb].value == "yes":
        cat["split_good"][jj] = True
    if rbuttons[rb].value == "no":
        cat["split_good"][jj] = False
    if rbuttons[rb].value == "unclear":
        cat["split_good"][jj] = None

Split detect
021 10001 yes
021 5006 yes
021 5007 yes
021 5008 yes
021 5013 no
022 10001 yes
022 10002 yes
022 10006 no
023 5014 yes
023 5028 yes
023 5032 yes
023 5038 yes
023 5042 yes
023 5046 yes
023 5047 yes
023 5059 yes
023 5061 yes
024 5006 yes
025 10002 yes
025 10026 no
025 10039 yes
025 5016 yes
025 5018 yes
025 5025 yes
025 5074 yes
025 5075 yes
026 5010 yes
026 5022 yes
027 10002 yes
027 10003 yes
032 10001 no
032 10002 yes
032 10010 no
032 10016 yes
032 10023 yes
032 5003 yes
032 5004 yes
032 5007 no
032 5013 yes
032 5026 yes
032 5047 yes
032 5055 no
032 5063 yes
032 5067 no
032 5078 yes
033 10003 yes
033 10004 no
033 10006 yes
033 10007 yes
034 5005 yes
037 10023 yes
037 5015 yes
037 5049 yes
042 5007 yes
042 5017 no
042 5050 yes
042 5058 yes
047 10003 no
047 5015 yes
047 5018 yes
047 5019 yes
047 5029 yes
047 5063 yes
052 5009 yes
052 5020 no
052 5068 yes
062 10009 yes
062 5003 yes
063 10011 no
063 10015 yes
063 5003 yes
063 5006 yes
072 10006 no
072 5001 yes
072 5002 yes
07

In [31]:
cat.write("../data/mmsf2outcube_COSMOSC_allifu.cat", format="ascii.ecsv", overwrite=True)